In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import scanpy as sc
sc.settings.n_jobs = 56
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [2]:
# Read the CellChat database
cellchat_db_df_file = '/home/xingjiepan/data/whole_brain/database/cellchat/cell_chat_mouse_interactions.csv'
cellchat_db_df = pd.read_csv(cellchat_db_df_file).set_index('Unnamed: 0')

# Generate the dictionary of ligand receptors
lr_dict = {}

for lr_name in cellchat_db_df.index:
    lr_name2 = cellchat_db_df.loc[lr_name, 'interaction_name_2']
    
    # Change to upper cases to avoid the inconsitant naming conventions
    ligands = lr_name2.split(' - ')[0].strip('(').strip(')').strip().upper()
    receptors = lr_name2.split(' - ')[1].strip('(').strip(')').strip().upper()
    
    lr_dict[lr_name] = {'ligands': ligands.split('+'), 'receptors': receptors.split('+')}
    
# Update the ligand and receptor names to mactch the sequencing data
name_map = {'INSSR': ['INSR'], 'CXCL4': ['PF4'], 'IL4R': ['IL4RA'], 'IL6R': ['IL6RA'],
            'GH1': ['GH'], 'IFNA': ['IFNA2'], 'CD45': ['PTPRC'], 'LXA4': ['FPR3'],
            'NGFA': ['KLK1B4'], 'JAM1': ['F11R'], 'H2-Q8': ['H2-Q6'], 'H2-T9': ['H2-T22'],
            'H2-T18': ['H2-T3'], 'H2-Q9': ['H2-Q7'], 'H2-L': ['H2-D1'], 'H2-D': ['H2-D1'],
           
            'VEGFR1': ['FLT1'], 'VEGFR2': ['KDR'], 'VEGFR3': ['FLT4'], 
            'VEGFR1R2': ['FLT1', 'KDR'], 'VEGFR2R3': ['KDR', 'FLT4'],
            
            'INHBABB': ['INHBA', 'INHBB'], 'INHABA': ['INHA', 'INHBA'], 
            'INHABB': ['INHA', 'INHBB'], 'INHB': ['INHBA'],
            
            'IL17AF': ['IL17A', 'IL17F']
           }

for lr_name in lr_dict:
    for gene in lr_dict[lr_name]['ligands']:
        if gene in name_map.keys():
            lr_dict[lr_name]['ligands'].remove(gene)
            lr_dict[lr_name]['ligands'] += name_map[gene]
    
    for gene in lr_dict[lr_name]['receptors']:
        if gene in name_map.keys():
            lr_dict[lr_name]['receptors'].remove(gene)
            lr_dict[lr_name]['receptors'] += name_map[gene]
            
# Get the all of the ligand receptor genes
all_lr_names = []
for lr_name in lr_dict:
    all_lr_names += lr_dict[lr_name]['ligands']
    all_lr_names += lr_dict[lr_name]['receptors']
    
all_lr_names = np.unique(all_lr_names)

# Get the imputation results

In [3]:
import os
import anndata

def load_imputation_data(imputation_path, genes=None, cell_types=None, verbose=False):
    '''Load the imputation results into an AnnData object with specified
    genes and cell types.'''
    # Load the table of gene partitions
    gene_partition_df = pd.read_csv(os.path.join(imputation_path, 'gene_partition.csv')).set_index('gene')
    
    # Change the gene names to upper cases
    # NOTE: This is only for the ligand-receptor analysis
    gene_partition_df.index = gene_partition_df.index.str.upper()

    # Define which genes to load
    if genes is None:
        genes = np.array(gene_partition_df.index)
       
    # Define which partitions to load
    partitions = np.unique(gene_partition_df['gene_partition'][gene_partition_df.index.isin(genes)])
       
    # Define which cell types to load
    ct_paths = sorted([f for f in os.listdir(imputation_path)
                       if os.path.isdir(os.path.join(imputation_path, f))])
    if not (cell_types is None):
        ct_paths = np.intersect1d(ct_paths, [ct.replace('/', '-').replace(' ', '_') for ct in cell_types])
   
    # Load the specified cell types
    adata_list = []
    for ctp in ct_paths:
        adata_ct_list = []
       
        # Load the specified genes
        for p in partitions:
            if verbose:
                print(f"Load {os.path.join(imputation_path, ctp, f'{p}.h5ad')}")
           
            adata_ct_p = anndata.read_h5ad(os.path.join(imputation_path, ctp, f'{p}.h5ad'))
            
            # Change the gene names to upper cases
            # NOTE: This is only for the ligand-receptor analysis
            adata_ct_p.var.index = adata_ct_p.var.index.str.upper()
            
            adata_ct_p = adata_ct_p[:, adata_ct_p.var.index.isin(genes)].copy()
            adata_ct_list.append(adata_ct_p)
         
        adata_list.append(anndata.concat(adata_ct_list, axis=1, merge='first'))
       
    return anndata.concat(adata_list, axis=0)

In [4]:
%%time
imputation_path = '/home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation'

adata = load_imputation_data(imputation_path, genes=all_lr_names, verbose=True)
adata.X = adata.X.toarray()

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/0.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/1.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/2.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/3.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/4.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/5.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ABC_NN/6.h5ad
Load /home/xingjiepan/data/whole_b

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/26.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/27.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/28.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/29.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/30.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ACB-BST-FS_D1_Gaba/31.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspa

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/17.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/18.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/19.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/20.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/21.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AD_Serpinb7_Glut/22.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expr

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/4.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/5.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/6.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/7.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/8.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN-SBPV-PVHd_Pdrm12_Gaba/9.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_f

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/26.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/27.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/28.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/29.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/30.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/AHN_Onecut3_Gaba/31.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expr

...

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/0.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/1.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/2.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/3.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/4.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/5.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Tanycyte_NN/6.h5ad

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/23.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/24.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/25.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/26.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/27.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VCO_Mafa_Meis2_Glut/28.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_w

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/15.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/16.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/17.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/18.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/19.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/VMH_Fezf1_Glut/20.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imput

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/6.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/7.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/8.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/9.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/10.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/11.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/Vip_Gaba/12.h5ad
Load /home/xingji

Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ZI_Pax6_Gaba/31.h5ad
Load /home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation/ZI_Pax6_Gaba/32.h5ad
CPU times: user 55min 20s, sys: 5min 59s, total: 1h 1min 19s
Wall time: 2h 17min 1s


In [5]:
adata

AnnData object with n_obs × n_vars = 9343481 × 961
    obs: 'integration_partition_transfer', 'integration_partition_confidence', 'subclass_label_transfer', 'subclass_label_confidence', 'cl_transfer', 'cl_confidence'

In [6]:
# Save the adata
adata.write('adate_imputed_lr_expression.h5ad')

# Generate adatas for ligands and receptors

In [7]:
# Only keep ligand receptor pairs whose all components are measured
measured_genes = set(adata.var.index)
lr_dict_measured = {}

for lr_name in lr_dict:
    if len(measured_genes.intersection(lr_dict[lr_name]['ligands'])) != len(lr_dict[lr_name]['ligands']):
        print(f'Miss ligands {lr_dict[lr_name]["ligands"]} for {lr_name}')
        continue
    if len(measured_genes.intersection(lr_dict[lr_name]['receptors'])) != len(lr_dict[lr_name]['receptors']):
        print(f'Miss receptors {lr_dict[lr_name]["receptors"]} for {lr_name}')
        continue
    
    lr_dict_measured[lr_name] = lr_dict[lr_name]

Miss ligands ['CXCL11'] for CXCL11_CXCR3
Miss ligands ['CXCL11'] for CXCL11_ACKR1
Miss ligands ['CXCL11'] for CXCL11_ACKR3
Miss receptors ['NCR3-PS'] for BAG6_NCR3-PS
Miss ligands ['NPNT1'] for NPNT1_ITGA8_ITGB1
Miss ligands ['NPNT2'] for NPNT2_ITGA8_ITGB1
Miss receptors ['GP COMPLEX'] for VWF_GP1BA_GP1BB_GP5_GP9
Miss ligands ['CD1D2'] for CD1D2_PIRA2
Miss ligands ['CD99'] for CD99_PILRA
Miss ligands ['CD99'] for CD99_CD99
Miss ligands ['H2-T10'] for H2-T10_CD8A
Miss ligands ['H2-BL'] for H2-BL_CD8A
Miss ligands ['GM10499'] for GM10499_CD8A
Miss ligands ['H2-T-PS'] for H2-T-PS_CD8A
Miss ligands ['H2-T10'] for H2-T10_CD8B1
Miss ligands ['H2-BL'] for H2-BL_CD8B1
Miss ligands ['GM10499'] for GM10499_CD8B1
Miss ligands ['H2-T-PS'] for H2-T-PS_CD8B1
Miss ligands ['H2-BL'] for H2-BL_KIR3DL1
Miss receptors ['KLRA'] for H2-D_KLRA
Miss receptors ['KLRA'] for H2-K1_KLRA
Miss ligands ['RAET1A'] for RAET1A_KLRK1
Miss ligands ['H60A'] for H60A_KLRK1
Miss ligands ['H2-EA'] for H2-EA_CD4


In [8]:
# Generate adatas for the ligand and receptor expression levels
X_ligand = np.zeros((adata.shape[0], len(lr_dict_measured)), dtype=np.float32)
X_receptor = np.zeros((adata.shape[0], len(lr_dict_measured)), dtype=np.float32)

lr_keys = list(lr_dict_measured)
for i, lr_pair in enumerate(lr_keys):
    X_ligand[:, i] = np.prod(adata.X[:, adata.var.index.isin(lr_dict_measured[lr_pair]['ligands'])], axis=1)
    X_receptor[:, i] = np.prod(adata.X[:, adata.var.index.isin(lr_dict_measured[lr_pair]['receptors'])], axis=1)
    
adata_ligand = anndata.AnnData(X=X_ligand, obs=adata.obs.copy(), 
                               var=pd.DataFrame({'lr_pair': lr_keys}).set_index('lr_pair'))
adata_receptor = anndata.AnnData(X=X_receptor, obs=adata.obs.copy(), 
                                 var=pd.DataFrame({'lr_pair': lr_keys}).set_index('lr_pair'))

In [9]:
# Save the data
adata_ligand.write('adate_imputed_lr_pair_ligand_expression.h5ad')
adata_receptor.write('adate_imputed_lr_pair_receptor_expression.h5ad')

# Get the cells that pass the quality filters

In [10]:
%%time
# Load the cell type labels
adata_filtered = sc.read_h5ad('/home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/adata_merfish_label_transfer.h5ad')
df_ct_labels = adata_filtered.obs.copy()
df_ct_labels = df_ct_labels[df_ct_labels['adjusted_subclass_label_confidence'] > 0.8]
df_ct_labels = df_ct_labels[df_ct_labels['subclass_label_transfer'] != 'LQ'].copy()
df_ct_labels.to_csv('cells_passed_quality_filter.csv')

CPU times: user 2min 13s, sys: 11.2 s, total: 2min 25s
Wall time: 5min 6s
